## Sub 52

In [49]:
# imports
import copy
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import xarray as xr  # noqa
from autoreject import AutoReject, get_rejection_threshold
from mne.time_frequency import psd_array_welch
from mne_bids import BIDSPath, read_raw_bids
from mne_connectivity import envelope_correlation, spectral_connectivity_time

from src.sugnet import amplitude_vector, dispersion_report, run_ica

# constants
# Network connectivit labels
yeo7 = {
    'N1': 'Visual',
    'N2': 'Somatomotor',
    'N3': 'DorsalAttention',
    'N4': 'VentralAttention',
    'N5': 'Limbic',
    'N6': 'Frontoparietal',
    'N7': 'Default',
    'mwall': 'Medial_Wall',
}

hemisferes = ['lh', 'rh']

# labels based on Yeo2011 atlas orders
networks = [yeo7[k]+'_'+ hemisferes[i]
            for k in yeo7.keys()
            for i in range(len(hemisferes))]

net_conn_labels = pd.DataFrame(index=networks, columns=networks)
net_conn_labels = net_conn_labels.apply(lambda x: x.index + '\N{left right arrow}' + x.name)
net_conn_labels = net_conn_labels.values[np.tril_indices(net_conn_labels.shape[0], k=-1)]

# sensor connectivity labels
# merge subject data with all data
# connectivity labels
epochs_fname = 'data/clean_data/sub-01_ses-01_task-baseline1_proc-clean_epo.fif'
epochs = mne.read_epochs(epochs_fname, preload=True)
epochs.drop_channels(ch_names=['M1', 'M2', 'EOG1', 'EOG2', 'ECG'])
ch_names = epochs.ch_names.copy()
sensor_conn_labels = pd.DataFrame(columns=ch_names, index=ch_names)
sensor_conn_labels = sensor_conn_labels.apply(lambda x: x.index + ' \N{left right arrow} ' + x.name)
sensor_conn_labels = sensor_conn_labels.values[np.triu_indices(sensor_conn_labels.shape[0], k=1)]


def preprocessing(raw,
                  subject,
                  task,
                  ref_chs=None,
                  filter_bounds=None,
                  ica_n_components=None,
                  autoreject=None,
                  ica_report=True,
                  n_job=-1):

    # initialize empty containers for storing amplitude vectors, rejection threshold, etc.
    pos = _make_montage()

    # set channels positions
    raw.set_montage(pos)

    # interpolate bad channels
    if raw.info['bads'] != []:
        raw.interpolate_bads()

    # filtering
    if filter_bounds is not None:
        raw.filter(
            l_freq=filter_bounds[0],
            h_freq=filter_bounds[1],
            n_jobs=n_job
            )

    # create amplitude vector before ICA
    ch_names = raw.ch_names

    # apply ICA, remove eog and ecg ICs using templates, and save the report
    raw = run_ica(raw, subject, task, n_components=ica_n_components, threshold=0.8, report=ica_report)

    # epoching (note: for creating epochs with mne.epochs, tmin and tmax should be specified!)
    epochs = mne.make_fixed_length_epochs(raw, duration=1, preload=True)
    del raw

    # autoreject
    if autoreject == 'global':
        # pick only eeg channels for getting rejection threshold:
        reject = get_rejection_threshold(epochs.copy().pick_types(eeg=True))
        epochs.drop_bad(reject=reject)

    if autoreject == 'local':
        ar = AutoReject()  # TODO consider setting random state
        epochs = ar.fit_transform(epochs)  # TODO check if this is ok to save the
        # fit_transformed data to the save object

    # rereferencing
    if ref_chs is not None:
        epochs.add_reference_channels(ref_channels='FCz')  # adding reference channel to the data
        epochs.set_eeg_reference(ref_channels=ref_chs)
    
    
    return epochs, reject


def _make_montage(path='data/raw/plb-hyp-live2131111.vhdr'):
    """
    Create a montage from barin vision raw data

    Parameters
    ----------
    path : str
        path to barinvision data header file

    """
    raw = mne.io.read_raw_brainvision(path, verbose=False, misc=['ECG'])
    raw.crop(1, 10)
    raw.load_data().set_channel_types({'ECG': 'ecg'})
    ch_names = copy.deepcopy(raw.info['ch_names'])
    ch_names.remove('ECG')
    pos_array = raw._get_channel_positions()

    pos_dict = dict(zip(ch_names, pos_array))
    pos = mne.channels.make_dig_montage(pos_dict)

    return pos


def get_session_data_for_sub(sessions_path, sub_id, task, session):
    """
    Get the session data for a given subject

    Parameters
    ----------
    sessions_path : str
        path to the session data
    sub_id : int
        subject id

    Returns
    -------
    session_data : pd.DataFrame
        session data for a given subject

    """
    condition = task + session
    session_data = pd.read_excel(sessions_path)
    session_data.set_index('bids_id', inplace=True)
    session_data = session_data.loc[sub_id]
    session_data = session_data[[f'hypnosis_depth_{session}', f'procedure_type_{session}',
                                 f'description_type_{session}']]
    session_data['session'] = session
    session_data['condition'] = condition
    session_data = session_data.to_frame().transpose()
    session_data.reset_index(inplace=True)
    session_data.index = [f'{sub_id}_{condition}']
    session_data.columns = ['bids_id', 'hypnosis_depth', 'procedure', 'description', 'session', 'condition']

    # reorder the columns
    session_data = session_data[['bids_id', 'condition', 'hypnosis_depth', 'procedure', 'description', 'session']]

    return session_data

def merge_with_all(path, sub_data):
    """
    Merge the data with the whole dataset

    Parameters
    ----------
    path : str
        path to the whole dataset
    sub_data : pd.DataFrame
        data for a single subject

    Returns
    -------
    merged : pd.DataFrame
        merged data

    """
    # open all data
    all_data = pd.read_csv(path, index_col=0)

    # update columns' types based on the whole dataset
    types = [all_data[col].dtype for col in all_data.columns[:6]]

    for col, t in zip(sub_data.columns[:6], types):
        sub_data[col] = sub_data[col].astype(t)
    
    # update subdata index to be one more than the last index of the whole dataset
    sub_data.index = [all_data.index[-1] + 1]

    # concatenate the new data to the whole dataset
    merged = pd.concat([all_data, sub_data])

    # update columns' names (replace "lowgamma" with "gamma")
    merged.columns = [col.replace('lowgamma', 'gamma') for col in merged.columns]

    return merged

def make_conn_df(conns, conn_labels, subject_id, task):
    # initiate an empty df
    conn_df = pd.DataFrame()
    for freq, conn in conns.items():
        conn_flat = conn[np.triu_indices(conn.shape[-1], k=1)]
        freq_labels = conn_labels + f' ({freq})'
        conn_df_ = pd.DataFrame(conn_flat, index=freq_labels, columns=[0]).transpose()
        conn_df = conn_df.join(conn_df_, how='outer')
        
    conn_df = conn_df.set_axis([f'{subject_id}_{task}'])
    
    return conn_df

Reading /Users/yeganeh/Codes/SugNet/data/clean_data/sub-01_ses-01_task-baseline1_proc-clean_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     999.00 ms
        0 CTF compensation matrices available
Not setting metadata
291 matching events found
No baseline correction applied
0 projection items activated


In [ ]:
# open data and preprocess
root = 'data/raw/sub-52/'
subject = '52'
task = 'experience2'

# An example of a single subject
subject_id = '214911relaxation.vhdr'
subject_path = root + subject_id

raw = mne.io.read_raw_brainvision(subject_path, preload=True, eog=('EOG1', 'EOG2'), misc=['ECG'])
raw.set_channel_types({'ECG': 'ecg'})

tmin = 2
tmax = 302
raw.crop(tmin=tmin, tmax=tmax)

### Preprocessing

In [ ]:
epochs, rejects = preprocessing(raw, subject, task, 'average', [1, 42], 30, 'global')
epochs.save(f'data/clean_data/sub-{subject}_ses-01_task-{task}_proc-clean_epo.fif', overwrite=True)

### source localization

In [4]:
sfreq = 512 # we resample the data to 512 Hz
frequencies = {'delta': (1, 4),
               'theta': (4, 8),
               'alpha': (8, 13),
               'beta': (13, 30),
               'lowgamma': (30, 42)  
}
# Helper functions
def create_raw_from_epochs(epochs):
    data = np.hstack(epochs.get_data())
    info = mne.create_info(ch_names=epochs.ch_names,
                           ch_types='eeg',
                           sfreq=epochs.info['sfreq'])

    raw = mne.io.RawArray(data, info)
    raw.set_channel_types({'ECG': 'ecg', 'EOG1': 'eog', 'EOG2': 'eog'})
    return raw

def make_forward():
    # fsaverage files
    fs_dir = Path('data/fsaverage')

    # The files live in:
    trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
    src = fs_dir / 'bem' / 'fsaverage-ico-4-src.fif' # use icosahedron4 with 6.2 mm source spacing
    src = mne.read_source_spaces(src)
    bem = fs_dir / 'bem' / 'fsaverage-5120-5120-5120-bem-sol.fif'
    
    path:Path = Path('data/clean_data')
    epochs_fname = path / 'sub-01_ses-01_task-baseline1_proc-clean_epo.fif'
    epochs = mne.read_epochs(epochs_fname, preload=True)
    epochs.resample(sfreq)

    # create raw object
    raw = create_raw_from_epochs(epochs)

    # insert channel positions
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)

    # forward solution (the same across all subjects)
    fwd = mne.make_forward_solution(raw.info, trans=trans, src=src,
                                    bem=bem, eeg=True, mindist=5.0, n_jobs=-1, verbose=False)
    del src, bem
    return fwd

def make_inverse_4baseline(subject: str,
                           fwd: mne.forward.forward.Forward,
                           path = Path('data/clean_data')):
    
    epochs_fname = path / f'{subject}_ses-01_task-baseline1_proc-clean_epo.fif'
    epochs = mne.read_epochs(epochs_fname, preload=True)
    epochs.resample(512)
    
    # create raw object
    raw = create_raw_from_epochs(epochs)

    # insert channel positions
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)
    
    raw.set_eeg_reference('average', projection=True)
    
    # covariance matrix
    cov = mne.compute_raw_covariance(raw, method='auto', cv=5, n_jobs=-1)

    # inverse operator
    inv = mne.minimum_norm.make_inverse_operator(raw.info, fwd, cov, verbose=False)
    
    return inv

def get_labels(subject: str,
                     task: str,
                     atlas_labels: list,
                     inv: mne.minimum_norm.inverse.InverseOperator,
                     path:Path = Path('data/clean_data'),
                     ):
    # open data
    epochs_fname = path / f'{subject}_ses-01_task-{task}_proc-clean_epo.fif'
    epochs = mne.read_epochs(epochs_fname, preload=True)
    epochs.resample(512)
    
    montage = mne.channels.make_standard_montage('standard_1020')
    epochs.set_montage(montage)
    
    epochs.set_eeg_reference('average', projection=True)

    stc = mne.minimum_norm.apply_inverse_epochs(epochs,
                                                inv,
                                                method='eLORETA',
                                                lambda2=1./9.,
                                                verbose=False)
    
    label_ts = mne.extract_label_time_course(stc,
                                             atlas_labels,
                                             inv['src'],
                                             return_generator=False,
                                             verbose=False)
    
    return label_ts

def get_connectivity(label_ts,
                     frequencies: dict,
                     sfreq: int = sfreq
):
    def bp_gen(label_ts, fmin, fmax, sfreq):
        for ts in label_ts:
            yield mne.filter.filter_data(ts, sfreq=sfreq, l_freq=fmin, h_freq=fmax)
    
    # each segment in labels_ts is 1 second
    # To compute the connectivity, we want segments that its lenght is about 30 seconds (or a bit less)
    label_continious = np.hstack(np.array(label_ts))
    label_ts = np.array_split(label_continious, 10, axis=1)
    
    conns = {} 
    for bp in frequencies.keys():
        conn_obj = envelope_correlation(bp_gen(label_ts, frequencies[bp][0], frequencies[bp][1], sfreq),
                                               orthogonalize='pairwise')
        conn = conn_obj.combine()
        conn = conn.get_data(output='dense')[..., 0]
        conns[bp] = conn
        
    return conns

In [ ]:
subject = 'sub-52'
task = 'experience2'
output_path_labels = f'data/parcellated_source_yeo7/{subject}_task-{task}_labels.npz'
output_path_conn = f'data/connectivities/{subject}_task-{task}_conn-corr_filtered.pkl'

# we will use Yeo2011 atlas (7 networks)
atlas_labels = mne.read_labels_from_annot('fsaverage',
                                          'Yeo2011_7Networks_N1000',
                                          subjects_dir='data/')

# forward solution (same across all subjects)
fwd = make_forward()

inv = make_inverse_4baseline('sub-52', fwd)
label_ts = get_labels(subject, task, atlas_labels, inv)
# np.savez_compressed(output_path_labels, labels=label_ts)

### PEC
#### at source level

In [28]:
# claculate or read connectivity data
subject = 'sub-52'
task = 'experience2'
output_path_conn = f'data/connectivities/{subject}_task-{task}_conn-corr_filtered.pkl'

# first check if the connectivity data exists in the path
if Path(output_path_conn).exists():
    with open(output_path_conn, 'rb') as f:
        conns = pickle.load(f)
    
else:
    conns = get_connectivity(label_ts, frequencies)
    with open(output_path_conn, 'wb') as f:
            pickle.dump(conns, f)

In [ ]:
conn_df = make_conn_df(conns, net_conn_labels, subject, task)
sess_data_sub52 = get_session_data_for_sub('data/behavioral_data/PLB_HYP_data_MASTER.xlsx', 'sub-52', 'experience2')
sess_data_sub52 = pd.concat([sess_data_sub52, conn_df], axis=1)
sess_data_sub52.index = [258]

merged = merge_with_all('data/classification_datasets/others/correlation_source.csv', sess_data_sub52)

# save merged data
# merged.to_csv('data/classification_datasets/correlation_source.csv')

#### at sensor level

In [2]:
subject = 'sub-52'
task = 'experience2'
epochs_fname = f'data/clean_data/{subject}_ses-01_task-{task}_proc-clean_epo.fif'

frequencies = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'lowgamma': (30, 42)  
}

# helper function
# Envelope Correlation at sensor level
def get_PEC_sensor(epochs,
                   frequencies: dict,
                   sfreq: float = 1000):

    def bp_gen(epochs, fmin, fmax, sfreq):
        for epoch in epochs:
            yield mne.filter.filter_data(epoch, sfreq=sfreq, l_freq=fmin, h_freq=fmax)
    
    # each segment in epochs is 1 second
    raw = np.hstack(epochs.get_data())
    
    # To compute the connectivity, we want segments that its length is about 30 seconds (or a bit less)
    epochs = np.array_split(raw, 10, axis=1)
    
    conns = {} 
    for bp in frequencies.keys():
        conn_obj = envelope_correlation(bp_gen(epochs, frequencies[bp][0], frequencies[bp][1], sfreq),
                                               orthogonalize='pairwise')
        conn = conn_obj.combine()
        conn = conn.get_data(output='dense')[..., 0]
        conns[bp] = conn
        
    return conns

In [3]:
# calculate or read connectivity data
connectivity_path = f'data/connectivities/correlation_sensor/{subject}_task-{task}_conn-corr_sensor_filtered.pkl'
if Path(connectivity_path).exists():
    with open(connectivity_path, 'rb') as f:
        conns = pickle.load(f)
else:
    print(f'>>>>> Calculating Connectivity Data for {subject}, {task} task <<<<<')
    # open epochs
    epochs = mne.read_epochs(epochs_fname, preload=True)

    # set montage
    montage = mne.channels.make_standard_montage('standard_1020')
    epochs.set_montage(montage)
        
    # surface laplacian
    epochs_csd = mne.preprocessing.compute_current_source_density(epochs)
    epochs_csd.drop_channels(ch_names=['M1', 'M2', 'EOG1', 'EOG2', 'ECG'])

    # compute connectivity
    conns = get_PEC_sensor(epochs_csd, frequencies, sfreq=1000)
    
    # # save connectivity
    # with open(connectivity_path, 'wb') as f:
    #     pickle.dump(conns, f)

In [23]:
conn_df = make_conn_df(conns, sensor_conn_labels, '52', task)
sess_data_sub52 = get_session_data_for_sub('data/behavioral_data/PLB_HYP_data_MASTER.xlsx', int(52), 'experience', '2')
sess_data_sub52 = pd.concat([sess_data_sub52, conn_df], axis=1)
sess_data_sub52.index = [258]

In [41]:
merged = merge_with_all('data/classification_datasets/others/correlation_sensor.csv', sess_data_sub52)
# save merged data
merged.to_csv('data/classification_datasets/correlation_sensor.csv')

### wPLI
#### at source level

In [ ]:
# constants
freqs = np.arange(1, 42, 1)
foi = np.array([
        [1, 4],
        [4, 8],
        [8, 13],
        [13, 30],
        [30, 42]
        ])
fmin = tuple(foi[:, 0])
fmax = tuple(foi[:, 1])

## bands
bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# define a function that get a numpy array and return a dataframe of FCs
def calculate_wpli(data, freqs, fmin, fmax, bands, subject, task, conn_labels, sfreq,
                   verbose=0, decim=20):
  conn = spectral_connectivity_time(data,
                                    freqs=freqs,
                                    method='wpli',
                                    fmin=fmin,
                                    fmax=fmax,
                                    sfreq=sfreq,
                                    faverage=True,
                                    average=True,
                                    verbose=verbose,
                                    decim=decim,
                                    n_cycles=5
                                  )

  # get data
  conn = conn.get_data(output='dense')
  
  # create a dataframe from a dictionary of connectivities in different bands
  temp = {}
  for i, b in enumerate(bands):
      temp[b] = conn[:, :, i][np.tril_indices(conn.shape[0], k=-1)]

  df = pd.DataFrame(temp, index=conn_labels).stack().reset_index()
  df['conn'] = df['level_0'] + '_' + df['level_1']
  
  return df.drop(columns=['level_0', 'level_1']).set_index('conn').T.set_index([pd.Index([subject + '_' + task])])

In [25]:
subject = 'sub-52'
task = 'experience2'
subject_path = f'data/parcellated_source_yeo7/{subject}_task-{task}_labels.npz'
label_ts = np.load(subject_path)['labels']
data = np.hstack(label_ts)

# To get 30-seconds time segments, we need to split source arrays into 10 segments.
# To make the array divisible by 10, we cut off a few data points (less than 10) at the end:
idx = data.shape[1] - data.shape[1]%10
data = data[:, :idx]
data = np.array(np.split(data, 10, axis=1))

# calculate wpli
df = calculate_wpli(data, freqs, fmin, fmax, bands, subject=subject, task=task, conn_labels=net_conn_labels,
                    verbose=0, decim=10, sfreq=512)

In [30]:
# get session data for a single subject
sess_data_sub52 = get_session_data_for_sub('data/behavioral_data/PLB_HYP_data_MASTER.xlsx', int(52), 'experience', '2')
df.index = ['52_experience2'] # rename connectivity index to make it match with session data index
sess_data_sub52 = pd.concat([sess_data_sub52, df], axis=1)
# # merge subject data with all data
merged = merge_with_all('data/classification_datasets/others/wpli_source.csv', sess_data_sub52)
# merged.to_csv('data/classification_datasets/wpli_source.csv')

#### at sensor level

In [ ]:
epochs_fname = f'data/clean_data/{subject}_ses-01_task-{task}_proc-clean_epo.fif'

epochs = mne.read_epochs(epochs_fname, verbose=0)
epochs.drop_channels(['M1', 'M2'])
epochs.pick_types(eeg=True)

# set montage
montage = mne.channels.make_standard_montage('standard_1020')
epochs.set_montage(montage)
print(epochs.info)

# surface laplacian
epochs_csd = mne.preprocessing.compute_current_source_density(epochs)
data = np.array(np.split(np.hstack(epochs_csd.get_data()), 10, axis=1))

df = calculate_wpli(data, freqs, fmin, fmax, bands, subject, task, sensor_conn_labels, sfreq=1000)

In [22]:
# get session data for a single subject
sess_data_sub52 = get_session_data_for_sub('data/behavioral_data/PLB_HYP_data_MASTER.xlsx', int(52), 'experience', '2')
df.index = ['52_experience2'] # rename connectivity index to make it match with session data index
sess_data_sub52 = pd.concat([sess_data_sub52, df], axis=1)
# # merge subject data with all data
merged = merge_with_all('data/classification_datasets/others/wpli_sensor.csv', sess_data_sub52)
merged.to_csv('data/classification_datasets/wpli_sensor.csv')

### Power
#### at source level

In [87]:
# constant variables to be used in the code
subject = 'sub-52'
task = 'experience2'
epochs_fname = f'data/clean_data/{subject}_ses-01_task-{task}_proc-clean_epo.fif'
source_path = f'data/parcellated_source_yeo7/{subject}_task-{task}_labels.npz'

# sampling rate
sfreq = 512

# frequency bands of interest
frequencies = {'delta': (1, 3.875),
               'theta': (4, 7.875),
               'alpha': (8, 12.875),
               'beta': (13, 30),
               'lowgamma': (30.125, 42) 
}
# bands = Bands(frequencies)

#helper functions
# get dataframe of powers for each frequency band
def get_power_bands_df(psds, freqs, labels=networks):
    
    # initiate a dictionary to store power from each band
    psds_bands = {}
    
    # average the power over each band
    for k in frequencies.keys():
        temp = psds[:, np.where((frequencies[k][0] <= freqs) & (freqs <= frequencies[k][1]) == True)[0]]
        psds_bands[k] = temp.mean(1)
    
    # create and return a dataframe with the power from each band across all networks
    df = pd.DataFrame.from_dict(psds_bands, orient='index', columns=labels).T.stack().reset_index()
    df['new_col'] = df['level_0'] + '_' + df['level_1']
    df.drop(['level_0', 'level_1'], axis=1, inplace=True)
    
    return df.set_index('new_col').T

In [37]:
label_ts = np.load(source_path)['labels']
  
# create a continuous data array from the parcellated source data
label_continious = np.hstack(np.array(label_ts))

# calculate psd on continuous data
psds, freqs = psd_array_welch(label_continious,
                              sfreq=sfreq,
                              fmin=1,
                              fmax=42,
                              n_fft=4096, # window size is 4096/512 = 8s
                              n_overlap=2048, # 50% overlap
                              n_jobs=1)

# extract periodic parameters
df_periodic = get_power_bands_df(psds, freqs=freqs)

# reindex the dataframe
df_periodic.index = [subject[4:] + '_' + task]

Effective window size : 8.000 (s)


In [50]:
# get session data for a single subject
sess_data_sub52 = get_session_data_for_sub('data/behavioral_data/PLB_HYP_data_MASTER.xlsx', int(52), 'experience', '2')
sess_data_sub52 = pd.concat([sess_data_sub52, df_periodic], axis=1)
# # merge subject data with all data
merged = merge_with_all('data/classification_datasets/others/power_source.csv', sess_data_sub52)
# merged.to_csv('data/classification_datasets/power_source.csv')

#### at sensor level

In [88]:
## channels name
epochs = mne.read_epochs('data/clean_data/sub-01_ses-01_task-baseline1_proc-clean_epo.fif', verbose=0)
ch_names = epochs.ch_names.copy()  # make sure to copy the list because it is mutable in place
[ch_names.remove(i) for i in ['M1', 'M2', 'EOG1', 'EOG2', 'ECG']]
all_channels = epochs.ch_names

# name of electrode groups
ba_patches = {'LF': ['Fp1', 'F3', 'F7', 'AF3', 'F1', 'F5'],
 'LC': ['C3', 'T7', 'FC1', 'FC3', 'FC5', 'C1', 'C5', 'FT7'],
 'LP': ['P3', 'P7', 'CP1', 'CP3', 'CP5', 'TP7', 'P1', 'P5'],
 'LO': ['O1', 'PO3'],
 'RF': ['Fp2', 'F4', 'F8', 'AF4', 'F2', 'F6',],
 'RC': ['C4', 'T8', 'FC2', 'FC4', 'FC6', 'C2', 'C6', 'FT8'],
 'RP': ['P4', 'P8', 'CP2', 'CP4', 'CP6', 'TP8', 'P2', 'P6'],
 'RO': ['O2', 'PO4'],
 'FZ': ['Fpz', 'Fz'],
 'CZ': ['Cz', 'FCz'],
 'PZ': ['Pz', 'CPz'],
 'OZ': ['POz', 'Oz', 'Iz'],
 'all': ch_names
}

# index of electrode groups
ba_patches_ind = {}
for k,v in ba_patches.items():
    temp = [all_channels.index(i) for i in v]
    ba_patches_ind[k] = temp

# frequency indces
freq = dict(delta=(0, 24),
            theta=(24, 56),
            alpha=(56, 96),
            beta=(96, 233),
            gamma=(233, 330))

############ Helper Functions ############
def calculate_psd(epochs_fname,
                  save=False):
    
    psds_dict = {}
    epochs = mne.read_epochs(epochs_fname)
    data = np.hstack(epochs.get_data())
    psds, _ = psd_array_welch(data,
                              sfreq=1000,
                              fmin=1,
                              fmax=42,
                              n_fft=8000,
                              verbose=0
                              )
    psds_dict[subject+'_'+task] = psds
    return psds_dict
    
def aggregate_psds(psds_dict, ba_patches_ind, freq):
    """Aggregate PSDs across channels and frequency bands."""
    # create a dataframe from aggreagated data, power in picovolts
    psds_agg = {}
    for k1, v1 in psds_dict.items():
        for k2, v2 in ba_patches_ind.items():
            for k3, v3 in freq.items():
                psds_agg[k1+'-'+k2+'_'+k3] = v1[v2].mean(0)[v3[0]:v3[1]].mean(0) * 10000 ** 3 #picovolts
    return psds_agg

In [91]:
psds_dict = calculate_psd(epochs_fname)
psds_agg = aggregate_psds(psds_dict, ba_patches_ind, freq)
psds_agg

Reading /Users/yeganeh/Codes/SugNet/data/clean_data/sub-52_ses-01_task-experience2_proc-clean_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     999.00 ms
        0 CTF compensation matrices available
Not setting metadata
296 matching events found
No baseline correction applied
0 projection items activated


/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_72017/3289573305.py:42: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = np.hstack(epochs.get_data())


{'sub-52_experience2-LF_delta': 4.892521786715944,
 'sub-52_experience2-LF_theta': 0.9543860275813599,
 'sub-52_experience2-LF_alpha': 1.2209368922444404,
 'sub-52_experience2-LF_beta': 0.4315413036384727,
 'sub-52_experience2-LF_gamma': 0.22736528887596727,
 'sub-52_experience2-LC_delta': 5.106679118335261,
 'sub-52_experience2-LC_theta': 1.031542644728039,
 'sub-52_experience2-LC_alpha': 1.0693203543227126,
 'sub-52_experience2-LC_beta': 0.44457140159289227,
 'sub-52_experience2-LC_gamma': 0.37459597368431974,
 'sub-52_experience2-LP_delta': 1.8522353085230137,
 'sub-52_experience2-LP_theta': 0.9009803332145465,
 'sub-52_experience2-LP_alpha': 2.0042830132649114,
 'sub-52_experience2-LP_beta': 0.6206558721276556,
 'sub-52_experience2-LP_gamma': 0.17280548352256098,
 'sub-52_experience2-LO_delta': 5.978224793763823,
 'sub-52_experience2-LO_theta': 1.4818193222901468,
 'sub-52_experience2-LO_alpha': 4.003589800633029,
 'sub-52_experience2-LO_beta': 0.6441777701275091,
 'sub-52_experien

In [90]:
psds_dict

{'sub-52_experience2': array([[8.92903172e-12, 7.84996205e-12, 6.58383492e-12, ...,
         3.44756695e-13, 2.82732296e-13, 2.38741841e-13],
        [4.98796117e-11, 4.04585345e-11, 4.15177696e-11, ...,
         4.93802499e-13, 5.32063204e-13, 5.82474023e-13],
        [4.07001780e-12, 3.70842352e-12, 4.57724059e-12, ...,
         8.83971160e-14, 9.24446141e-14, 6.66867137e-14],
        ...,
        [9.92837041e-12, 8.00557987e-12, 5.58430869e-12, ...,
         7.02463258e-14, 5.20172497e-14, 5.74307540e-14],
        [1.42892348e-11, 1.57358464e-11, 1.15148502e-11, ...,
         1.02368198e-13, 9.46011765e-14, 1.42115199e-13],
        [3.07785375e-12, 2.82826174e-12, 2.05873624e-12, ...,
         6.37379717e-14, 6.21788265e-14, 5.92624144e-14]])}

In [ ]:
df = pd.read_excel('data/PLB_HYP_data_MASTER.xlsx')
id_vars=['timestamp_trigger_4_recording_1',
 'timestamp_trigger_4_recording_2',
 'timestamp_trigger_4_recording_3',
 'timestamp_trigger_4_recording_4',
 'timestamp_trigger_5_recording_1',
 'timestamp_trigger_5_recording_2',
 'timestamp_trigger_5_recording_3',
 'timestamp_trigger_5_recording_4',
 'timestamp_trigger_6_recording_1',
 'timestamp_trigger_6_recording_2',
 'timestamp_trigger_6_recording_3',
 'timestamp_trigger_6_recording_4',
 'timestamp_trigger_7_recording_1',
 'timestamp_trigger_7_recording_2',
 'timestamp_trigger_7_recording_3',
 'timestamp_trigger_7_recording_4',
 ]

df = df.melt(id_vars=id_vars,
 value_vars=['procedure_type_1',
 'procedure_type_2',
 'procedure_type_3',
 'procedure_type_4'])

df['session'] = df['variable'].apply(lambda x: x.split('_')[-1])
df['timestamp'] = df.apply(lambda r: r['timestamp_trigger_4_recording_'+r['session']], axis=1)